# Z3 : un problème ? Pas de problème !

In [1]:
from z3 import *  # la ligne qui ajoute 10 points de style

## Avant-propos

z3 est un **résolveur de contraintes** créé par Microsoft Research

# Le lycéen fainéant

Un lycéen qui n'en peut plus des Maths mais qui a des notions de Python décide de scripter la résolution de son exercice.

L'énoncé est le suivant :
"*Trouver des valeurs de a, b et c validant l'expression a + 2b + 3c = a + c tels que a != b != c a,b,c étant entiers*"

Le lycéen commence donc par **définir ses variables** avec z3

In [2]:
a = Int('a')  # définit une variable entière de valeur inconnue qui sera nommée comme 'a'
b = Int('b')
c = Int('c')

solver = Solver()  # déclare une instance du solveur de contraintes

Il explicite ensuite les **contraintes** de l'exercices

In [3]:
# a != b != c
solver.add(Distinct(a, b, c))
solver.add(a + 2 * b + 3 * c == a + c)

Il résout son exercice

In [4]:
solver.check()

sat

*sat* signifie que les contraintes ont été satisfaites, autrement dit que le solveur a trouvé des solutions aux contraintes !

In [5]:
solver.model()

[b = -1, a = -2, c = 1]

```-2 != -1 != 1```
et
```-2 + -1 * 3 + 1 * 3 = -2 + 1```
<=>
```-2 = -2```
l'exercice est résolu !

# Logique : opérations binaires

Ok maintenant le lycéen va jouer à Minecraft. Il va sur un serveur de mini-jeux et joue à un jeu de redstone. 
Pour ceux qui ne voient pas du tout ce qu'est la redstone, imaginez un circuit composés d'entrées : les leviers (ON => 1, OFF => 0) et des portes logiques (AND, OR, XOR).

Il a devant lui le circuit que l'on peut schématiser comme suit :

```
Levier 1 ===========
                   |= AND =
Levier 2 ===========      |= XOR =====
                          |          |
Levier 3 ==================          |== AND==>>
                                     |
Levier 4 =============================

"=" et "|" représentent le circuit
">>" représente la sortie
```

Le but du challenge, c'est d'activer les bons leviers pour qu'en sortie, on obtienne 1.
Ayant un terrible enthousiasme pour z3, il décide d'utiliser ce framework pour résoudre l'épreuve au lieu de réfléchir 1 sec.

In [6]:
# on définit les leviers (sachant que True == 1 == levier activé)
lev1 = Bool('lev1')
lev2 = Bool('lev2')
lev3 = Bool('lev3')
lev4 = Bool('lev4')

# on instancie notre solveur
solver_minecraft = Solver()

In [7]:
# on définit la logique de notre circuit
op1 = And(lev1, lev2)
op2 = Xor(op1, lev3)
op3 = And(op2, lev4)

solver_minecraft.add(op3)
# Remarque : solver_minecraft.add(op3) == solver_minecraft.add(op3 == True)

In [8]:
solver_minecraft.check()
solver_minecraft.model()

[lev1 = False, lev2 = False, lev3 = True, lev4 = True]

Une solution à ce problème est donc
```
    OFF  ==========
                   |= AND =
    OFF  ==========       |= XOR =====
                          |          |
     ON  ==================          |== AND==>> 1
                                     |
     ON  =============================
```

# On s'énerve un petit coup

Bon maintenant on arrête l'histoire du lycéen et on se concentre sur de vrais problèmes.

### Sudoku basique

Rappel : le but du sudoku est de placer dans les cases des nombres, chaque nombre devant être unique sur sa ligne ou colonne.
Normalement il y a également la contrainte des zones que l'on va retirer ici pour commencer cette partie en douceur.

In [9]:
def solve_sudoku(array: list) -> list or None:
    """ solves the sudoku game defines in map
    
    each element of the array will be transformed to z3 Int and
    constraints are added to specify that each line and colomn
    must not contain equal numbers
    
    :array is a list of sudoku game lines
    :returns the completed sudoku
    """
    solver = Solver()
    # double tableau ou chaque élément va être une instance Int()
    ints = []
    for line in range(len(array)):
        ints.append([])
        for element in range(len(array[line])):
            # on nomme chaque élément iCOLONNE_LIGNE
            ints[-1].append(Int(f"i{element}_{line}"))
            # tous les nombres doivent être compris entre 1 et le nombre d'éléments sur la ligne
            solver.add(And(ints[-1][-1] <= len(array[line]), ints[-1][-1] >= 1))
            # si l'élément actuel est initialisé, on ajoute la contrainte élément == valeur d'initialisation
            if array[line][element] != 0:
                solver.add(ints[-1][-1] == array[line][element])
        # chaque élément de la ligne qui vient d'être ajoutée doit être unique
        solver.add(Distinct(ints[-1]))
    # on créé des tableaux pour chaque colonne et on précise que chaque élément sur la colonne doit être unique
    for column in range(len(array)):
        check_colomns = []
        for line in range(len(ints)):
            check_colomns.append(ints[line][column])
        solver.add(Distinct(check_colomns))
    # on solve les contraintes
    c = solver.check()
    if c is unsat:
        print("No solution found, are you sure the puzzle can be solve ?")
        return None
    return solver.model()


game = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 2]
]

res = solve_sudoku(game)
print(res)

[i0_4 = 1,
 i1_1 = 5,
 i2_0 = 5,
 i3_2 = 3,
 i0_3 = 5,
 i2_3 = 3,
 i1_4 = 3,
 i2_4 = 4,
 i0_2 = 2,
 i3_3 = 4,
 i4_1 = 3,
 i3_4 = 5,
 i4_3 = 1,
 i1_2 = 4,
 i0_0 = 3,
 i4_0 = 4,
 i3_0 = 2,
 i2_1 = 2,
 i4_2 = 5,
 i1_0 = 1,
 i3_1 = 1,
 i1_3 = 2,
 i0_1 = 4,
 i4_4 = 2,
 i2_2 = 1]


Le résultat final est :
```
    5 4 3 2 1
    1 2 5 4 3
    2 3 1 5 4
    4 1 2 3 5
    3 5 4 1 2
```

### Collision de hashs

Un hash est une fonction mathématique qui :
*  n'est **pas réversible** hash(a) = b mais pas de fonction telle que hash⁻¹(b) = a !
*  pour chaque valeur de a, il existe une **unique** valeur b


Il existe plusieurs fonctions de hash, comme MD5, SHA(1,2,3) ...
Un hash permet par exemple de stocker des mots de passe de manière safe : si on se fait voler la base de données, l'attaquant ne peut pas retrouver les mots de passe à partir des hash.


Certaines fonctions de hash (comme MD5) sont vulnérables aux **attaques par collision**. Le but d'une collision est de trouver deux inputs différents dont le résultat du hash est identique.
hash(a) = b = hash(c) and a != c => collision

In [24]:
from hashlib import md5

def H(input_bytes):
    '''This idea comes directly from awesome James Forshaw, read his blogpost:
    http://tyranidslair.blogspot.co.uk/2014/09/generating-hash-collisions.html'''
    h = 0
    for byte in input_bytes:
        h = h * 31 + ZeroExt(24, byte)
    return h

def print_res(solver, size):
    m = solver.model()
    result = sorted([[d, m[d]] for d in m], key = lambda x: str(x[0]))
    res1 = ''
    xd =[[], []]
    for i in range(size):
        res1 += chr(result[i][1].as_long())
        xd[0].append(result[i][1])
    res2 = ''
    for i in range(size):
        res2 += chr(result[size + i][1].as_long())
        xd[1].append(result[i][1])
    print(f"res1 = \"{res1}\"", f"res2 = \"{res2}\"")

def check_collision(size: int, hash_fn: object) -> bool:
    """ checks if the given hash fonction can be exploited with a collision attack on input of given size
    
    :size size of the inputs to check collision
    :hash_fn hash function that must take as parameter a byte array
    :returns True if collision was found with given size
    """
    # on crée deux inputs composés de valeurs de 8bits (un octet)
    solver = Solver()
    input1 = [BitVec(f"input1_{i}", 8) for i in range(size)]
    input2 = [BitVec(f"input2_{i}", 8) for i in range(size)]
    for i in input1:
        solver.add(And(i >= 30, i <= 126))
    for i in input2:
        solver.add(And(i >= 30, i <= 126))
    solver.add(H(input1) == H(input2))
    solver.add(input1 != input2)
    if solver.check():
        print_res(solver, size)
        return True
    return False

check_collision(7, H)

res1 = "k{Sfy" res2 = "{LtAi("


True

### Chiffrement par xor répété : on va casser Vigenère

**Point crypto rapide**
*  par définition du xor, ```a ^ b = c``` <=> ```a ^ c == b```
*  c'est le principe de base du chiffrement symétrique : a est la clé, b le message en clair et c le message chiffré
*  "aaa" chiffré/xoré avec la clé "." donne "OOO" et "OOO" xoré avec "." donne "aaa"
*  pour une clé à plus d'un octet, on fait un "xor répété" : "iamlo" chiffré avec "abc" revient à faire :
```
"i" ^ "a"
"a" ^ "b"
"m" ^ "c"
"l" ^ "a"
"o" ^ "b"
```
Cette méthode de chiffrement est dite de "substitution polyalphabétique" ou chiffrement de Vigenère.

**L'exercice**

On reçoit le message chiffré suivant : 

### Le problèmes des nqueens

L'exercice des nqueens est un peu le hello world de l'algorithmie. Le but est très simple : on prend un échiquier  de n*n cases et le but est d'y placer le maximum de reines possibles sans qu'elles puissent s'attaquer. Pour rappel une reine peut attaquer sur toute une ligne, colonne ou diagonale.

On part donc du constat qu'il est impossible d'avoir deux reines sur la même ligne.

In [12]:
queens = []
n = 8  # n est le nombre de lignes et de colones
nqueens_solver = Solver()
for i in range(n):
    # on nomme chaque reine qi avec i le numéro de leur ligne respective
    queens.append(Int(f"q{i+1}"))
    # on spécifie que les reines doivent être sur l'échiquier
    nqueens_solver.add(And(queens[-1] >= 1, queens[-1] <= n))
# on précise que aucune reine ne peut se retrouver sur la même colonne
nqueens_solver.add(Distinct(queens))
check_diags = []
j = 0
for i in range(j):
    for j in range(n):
        check_diags.append(
            If (i == j, True, And(queens[i] - queens[j] != i - j, queens[i] - queens[j] != j - i))
        )
nqueens_solver.add(check_diags)

nqueens_solver.check()
nqueens_solver.model()

[q5 = 3,
 q7 = 5,
 q3 = 8,
 q8 = 4,
 q2 = 7,
 q4 = 1,
 q6 = 2,
 q1 = 6]